In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# Imports 

In [6]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from transformers import AutoTokenizer, DistilBertModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
import sys
#sys.path.append('/lfs/1/sahaana/enrichment/enrich/utils')
sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
 
    
from embedding_datasets import DeepMatcherDataset, EmberEvalDataset
from embedding_models import TripletSingleBERTModel
from embedding_utils import param_header, tokenize_batch  
from embedding_runner import train_model, eval_model
#from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   
#from models import BatchedTripletSingleTowerModel, BatchedTripletSingleBERTModel
#from model_runner import train_model, eval_model
from knn_utils import FaissKNeighbors, knn_deepmatcher_recall #, knn_matching_accuracy, find_perfect_recall

In [ ]:
%load_ext autoreload
%autoreload 2

# Load Processed (cleaned, aligned) Datasets

In [6]:
left = '/lfs/1/sahaana/enrichment/data/deepmatcher/company_exp_data/tableA_processed.pkl'
right = '/lfs/1/sahaana/enrichment/data/deepmatcher/company_exp_data/tableB_processed.pkl'

left = pd.read_pickle(left)
right = pd.read_pickle(right)

In [7]:
left

,content,merged_all
id,,
http://www.wikidata.org/entity/Q6843249_wiki,and the outdoors type private founded 1977 hea...,[SEP] content and the outdoors type private ...
http://www.wikidata.org/entity/Q7243708_wiki,verification . please help improve this articl...,[SEP] content verification . please help imp...
http://www.wikidata.org/entity/Q4047803_wiki,filename extension . bik . bk2 type code bink ...,[SEP] content filename extension . bik . bk2...
http://www.wikidata.org/entity/Q834077_wiki,might be slanted towards recent events . pleas...,[SEP] content might be slanted towards recen...
http://www.wikidata.org/entity/Q4836856_wiki,sources . please help improve this article by ...,[SEP] content sources . please help improve ...
...,...,...
http://www.wikidata.org/entity/Q5059736_wiki,knowledge societies type private industry expe...,[SEP] content knowledge societies type priva...
http://www.wikidata.org/entity/Q5973259_wiki,the subject in a subjective manner without imp...,[SEP] content the subject in a subjective ma...
http://www.wikidata.org/entity/Q20644661_wiki,founded june 2009 headquarters cambridge canad...,[SEP] content founded june 2009 headquarters...


In [8]:
right

,content,merged_all
id,,
http://www.wikidata.org/entity/Q991188_official,and may not be able to properly display our we...,[SEP] content and may not be able to properl...
http://www.wikidata.org/entity/Q19880587_official,partnerships people collective gs4g contact of...,[SEP] content partnerships people collective...
http://www.wikidata.org/entity/Q6333706_official,quantity is of types of schools that function ...,[SEP] content quantity is of types of school...
http://www.wikidata.org/entity/Q6717757_official,have an account yet compare accounts or start ...,[SEP] content have an account yet compare ac...
http://www.wikidata.org/entity/Q4836856_official,pacificbw lngbw vlccbw chemical tankersbw dry ...,[SEP] content pacificbw lngbw vlccbw chemica...
...,...,...
http://www.wikidata.org/entity/Q6823302_official,troupe that integrates all of what second life...,[SEP] content troupe that integrates all of ...
http://www.wikidata.org/entity/Q6336561_official,good website design 08 19 2016 charlotte hanse...,[SEP] content good website design 08 19 2016...
http://www.wikidata.org/entity/Q7390286_official,change browser settings to disable cookies if ...,[SEP] content change browser settings to dis...


# Prepping Data Indices

In [7]:
train_df = '/lfs/1/sahaana/enrichment/data/deepmatcher/company_exp_data/supervision_train.pkl'
train_df = pd.read_pickle(train_df)

test_df = '/lfs/1/sahaana/enrichment/data/deepmatcher/company_exp_data/supervision_test.pkl'
test_df = pd.read_pickle(test_df)

In [8]:
test_df

,ltable_id,rtable_id,label
0,http://www.wikidata.org/entity/Q7956130_wiki,http://www.wikidata.org/entity/Q7956130_official,1
1,http://www.wikidata.org/entity/Q1629768_wiki,http://www.wikidata.org/entity/Q3046141_official,0
2,http://www.wikidata.org/entity/Q6926815_wiki,http://www.wikidata.org/entity/Q18148806_official,0
3,http://www.wikidata.org/entity/Q5190093_wiki,http://www.wikidata.org/entity/Q7258424_official,0
4,http://www.wikidata.org/entity/Q19630497_wiki,http://www.wikidata.org/entity/Q2286590_official,0
...,...,...,...
22498,http://www.wikidata.org/entity/Q7280746_wiki,http://www.wikidata.org/entity/Q7912375_official,0
22499,http://www.wikidata.org/entity/Q6464979_wiki,http://www.wikidata.org/entity/Q6464979_official,1
22500,http://www.wikidata.org/entity/Q17510971_wiki,http://www.wikidata.org/entity/Q3116885_official,0
22501,http://www.wikidata.org/entity/Q404542_wiki,http://www.wikidata.org/entity/Q1423707_official,0


## Model H Params

In [14]:
epochs = 1
batch_size = 8
final_size = 200
lr = .00001
tl_margin = 1.0
tl_p = 2
pool_type = "CLS"
column = "merged_all"
shuffle = True
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_max_length = 512
compute_val = False

bert_path='/lfs/1/sahaana/enrichment/ember/pretraining/models/abt_buy_exp_data-uncased-masked-ALL-BM25'
bert_model = DistilBertModel.from_pretrained(bert_path, return_dict=True)

model_name = f'abt_buy_exp_data-uncased-masked-ALL-BM25'

## Init and Run Model

In [15]:
triplet_loss = nn.TripletMarginLoss(margin=tl_margin, p=tl_p)
losses = []
val_losses = []
model = TripletSingleBERTModel(final_size, pool_type, bert_path)
optimizer = optim.AdamW(model.parameters(), lr=lr)#optim.SGD(model.parameters(), lr=lr)

In [17]:
model_checkpoint = "/lfs/1/sahaana/enrichment/ember/embedding/models/company_exp_data-distilbert-base-uncased-single-triplet-67596-1/batch_size-8-final_size-200-opt_lr-1e-05-pooling-CLS-epochs-1-train-67596/07-30-22-02-21"

model.load_state_dict(torch.load(model_checkpoint))

<All keys matched successfully>

# k-NN Model Performance

In [21]:
left_data = DataLoader(EmberEvalDataset(left, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )
right_data = DataLoader(EmberEvalDataset(right, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )

In [22]:
left_index, left_embeddings = eval_model(model, tokenizer, left_data, tokenizer_max_length=512)
right_index, right_embeddings = eval_model(model, tokenizer, right_data, tokenizer_max_length=512)

In [24]:
knn = FaissKNeighbors(k=30)
knn.fit(right_embeddings)
neib = knn.kneighbors(left_embeddings)

In [42]:
top_index, supervision = knn_deepmatcher_recall(neib[0], neib[1], test_df, left_index, right_index, k = 1)

In [43]:
top_index

defaultdict(set,
            {0: {'http://www.wikidata.org/entity/Q6843249_official'},
             1: {'http://www.wikidata.org/entity/Q7243708_official'},
             2: {'http://www.wikidata.org/entity/Q4047803_official'},
             3: {'http://www.wikidata.org/entity/Q7351972_official'},
             4: {'http://www.wikidata.org/entity/Q4836856_official'},
             5: {'http://www.wikidata.org/entity/Q6336475_official'},
             6: {'http://www.wikidata.org/entity/Q7079486_official'},
             7: {'http://www.wikidata.org/entity/Q287537_official'},
             8: {'http://www.wikidata.org/entity/Q6748573_official'},
             9: {'http://www.wikidata.org/entity/Q5065768_official'},
             10: {'http://www.wikidata.org/entity/Q16252590_official'},
             11: {'http://www.wikidata.org/entity/Q7671060_official'},
             12: {'http://www.wikidata.org/entity/Q4633173_official'},
             13: {'http://www.wikidata.org/entity/Q4761638_official'},

In [44]:
supervision

array([['http://www.wikidata.org/entity/Q7956130_wiki',
        'http://www.wikidata.org/entity/Q7956130_official', 1],
       ['http://www.wikidata.org/entity/Q1629768_wiki',
        'http://www.wikidata.org/entity/Q3046141_official', 0],
       ['http://www.wikidata.org/entity/Q6926815_wiki',
        'http://www.wikidata.org/entity/Q18148806_official', 0],
       ...,
       ['http://www.wikidata.org/entity/Q17510971_wiki',
        'http://www.wikidata.org/entity/Q3116885_official', 0],
       ['http://www.wikidata.org/entity/Q404542_wiki',
        'http://www.wikidata.org/entity/Q1423707_official', 0],
       ['http://www.wikidata.org/entity/Q2373202_wiki',
        'http://www.wikidata.org/entity/Q2373202_official', 1]],
      dtype=object)

In [55]:
knn_deepmatcher_recall_tes(neib[0], neib[1], test_df, left_index, right_index, k = 1)

(0.9955049445609829,
 0.5890070921985816,
 0.7401136237050241,
 None,
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,


In [54]:
from sklearn.metrics import precision_recall_fscore_support
def knn_deepmatcher_recall_tes(dists: np.array, 
                               neibs: np.array, 
                               supervision: pd.DataFrame,
                               left_indexing: np.array,
                               right_indexing: np.array,
                               k: int = None,
                               thresh: float = None):
    neibs = right_indexing[neibs]
    if k is not None:
        l, r = np.where(dists <= np.max(dists[:,:k], axis=1)[:,None]) ## to get all equidistant mins
    else:
        l, r = np.where(dists <= thresh) 
    
    top_index = defaultdict(set)
    for i,j in zip(l,r):
        top_index[left_indexing[i]].add(neibs[i,j])
    
    predicted = []
    supervision = supervision.to_numpy()
    true = supervision[:,2].astype(int)
    for left, right, label in supervision:
        if right in top_index[left]:
            predicted += [1]
        else:
            predicted += [0]
    #return top_index, supervision
    return *precision_recall_fscore_support(true, predicted, average = 'binary'), predicted

In [219]:
np.where(neib[0][720] == np.min(neib[0][720]))

(array([0, 1, 2]),)

In [237]:
np.where(neib[0] == np.min(neib[0], axis=1)[:,None],)

(array([   0,    1,    2, ..., 1078, 1079, 1080]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [233]:
np.min(neib[0], axis=1)[:,None].shape

(1081, 1)

In [241]:
a, b = np.where(neib[0] == np.min(neib[0], axis=1)[:,None])

In [262]:
top_matches = defaultdict(set)
top_index = defaultdict(set)
for i,j in zip(a,b):
    top_matches[i].add(j)
    top_index[i].add(neib[1][i,j])

In [263]:
top_matches

defaultdict(set,
            {0: {0},
             1: {0},
             2: {0},
             3: {0},
             4: {0},
             5: {0},
             6: {0},
             7: {0},
             8: {0},
             9: {0},
             10: {0},
             11: {0},
             12: {0},
             13: {0},
             14: {0},
             15: {0},
             16: {0},
             17: {0},
             18: {0},
             19: {0},
             20: {0},
             21: {0},
             22: {0},
             23: {0},
             24: {0},
             25: {0},
             26: {0},
             27: {0},
             28: {0},
             29: {0},
             30: {0},
             31: {0},
             32: {0},
             33: {0},
             34: {0},
             35: {0},
             36: {0},
             37: {0},
             38: {0},
             39: {0},
             40: {0},
             41: {0},
             42: {0},
             43: {0},
             44: {0},
   

In [264]:
top_index

defaultdict(set,
            {0: {53},
             1: {657},
             2: {33},
             3: {195},
             4: {153},
             5: {191},
             6: {213},
             7: {161},
             8: {37},
             9: {179},
             10: {34},
             11: {35},
             12: {58},
             13: {197},
             14: {44},
             15: {56},
             16: {223},
             17: {845},
             18: {211},
             19: {40},
             20: {40},
             21: {196},
             22: {62},
             23: {491},
             24: {42},
             25: {21},
             26: {201},
             27: {47},
             28: {48},
             29: {43},
             30: {43},
             31: {643},
             32: {50},
             33: {198},
             34: {236},
             35: {199},
             36: {228},
             37: {316},
             38: {192},
             39: {984},
             40: {54},
             41: {811},
    

In [275]:
for i,j,k in train_df.to_numpy():
    print(i)
    break

730


In [ ]:

for k in range(30):
    print(k, *knn_matching_accuracy(neib[1], k, train_idx, test_idx))

In [168]:
test_df

,ltable_id,rtable_id,label
0,445,910,0
1,719,812,0
2,550,664,0
3,942,949,1
4,643,636,0
...,...,...,...
1911,522,936,0
1912,720,799,1
1913,385,454,0
1914,167,569,0


In [172]:
neib[1][720]

array([ 578, 1084, 1045,  774,  551,  610,  558,  780, 1041,  879,  799,
        801,  800,  184, 1015,  916,   35,  709,  217, 1026,  762,  460,
        878,  557,  798,  177,  773,  772,  763,  970])

In [174]:
left.loc[720]

id                                                           720
name           lg 2.0 cu . ft. over-the-range black microwave...
description    lg 2.0 cu . ft. over-the-range black microwave...
price                                                        NaN
merged_all      [SEP]  name lg 2.0 cu . ft. over-the-range bl...
Name: 720, dtype: object

In [181]:
right.loc[799]

id                                                           799
name                lg 2.0 cu.ft . over the range microwave oven
description                                                  NaN
price                                                        NaN
merged_all      [SEP]  name lg 2.0 cu.ft . over the range mic...
Name: 799, dtype: object

In [191]:
neib[1][720]

array([ 801,  799,  800,  780,  217,  787,  789,  788,  774,  783,  786,
        785,  784,  782,  773,  772,  766,  781,  423,  426,  779,  777,
        453,  765,  770,  768,  769,  767, 1045, 1084])

In [194]:
neib[0][720]

array([10.280792 , 10.280792 , 10.280792 , 10.359413 , 12.4604   ,
       13.335747 , 13.335747 , 13.335747 , 13.568726 , 14.973106 ,
       14.973106 , 14.973106 , 14.973106 , 14.973106 , 15.100166 ,
       15.118561 , 15.217945 , 15.246048 , 15.458317 , 15.641705 ,
       15.8689575, 15.987331 , 16.198994 , 16.24091  , 16.743385 ,
       16.743385 , 16.743385 , 16.74339  , 18.32719  , 18.65799  ],
      dtype=float32)

In [196]:
train_df[train_df['ltable_id'] == 720]

,ltable_id,rtable_id,label
823,720,1079,0
1118,720,780,0
1317,720,349,0
3376,720,800,0


In [197]:
test_df[test_df['ltable_id'] == 720]

,ltable_id,rtable_id,label
89,720,798,0
947,720,812,0
1190,720,811,0
1912,720,799,1


In [257]:
left.loc[719]['name'], left.loc[719]['description']

('lg 2.0 cu . ft. over-the-range white microwave oven lmvm2085wh',
 'lg 2.0 cu . ft. over-the-range white microwave oven lmvm2085wh warming lamp glide & spin sliding tray with turntable sensor cook feature 300 cfm exhaust system horizontal keypad elegant hidden vent wideview window halogen cooktop lighting white finish')

In [258]:
left.loc[722]['name'], left.loc[722]['description']

('lg 2.0 cu . ft. over-the-range stainless steel microwave oven lmvm2085ss',
 'lg 2.0 cu . ft. over-the-range stainless steel microwave oven lmvm2085ss warming lamp glide & spin sliding tray with turntable sensor cook feature 300 cfm exhaust system horizontal keypad elegant hidden vent wideview window halogen cooktop lighting stainless steel finish')

In [209]:
left.loc[720]['name'], left.loc[720]['description']

('lg 2.0 cu . ft. over-the-range black microwave oven lmvm2085bk',
 'lg 2.0 cu . ft. over-the-range black microwave oven lmvm2085bk warming lamp glide & spin sliding tray with turntable sensor cook feature 300 cfm exhaust system horizontal keypad elegant hidden vent wideview window halogen cooktop lighting black finish')

In [211]:
right.loc[799]['name'], right.loc[799]['description']

('lg 2.0 cu.ft . over the range microwave oven', nan)

In [212]:
right.loc[800]['name'], right.loc[800]['description']

('lg 2.0 cu.ft . over the range microwave oven', nan)

In [259]:
right.loc[801]['name'], right.loc[801]['description']

('lg 2.0 cu.ft . over the range microwave oven', nan)